### Test Code



#### Checking the input data

In [5]:
import pandas as pd
import boto3
import json
import os

In [6]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','SECRET_ACCESS_KEY')

KEY = config.get('AWS','KEY_ID')
SECRET = config.get('AWS','SECRET_ACCESS_KEY')

In [7]:
import boto3

s3 = boto3.resource('s3',
                     region_name='us-west-2',
                     aws_access_key_id=KEY,
                     aws_secret_access_key= SECRET
                    )


In [13]:
sampleDbBucket =  s3.Bucket("udacity-dend")

# Iterate over bucket objects starting with "ssbgz" and print
i = 0
for obj in sampleDbBucket.objects.all():
    if i==150:
        break
    i+=1
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/data-pipeline-dataset-creation.zip')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/1/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/10/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/11/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/12/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/2/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/3/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/partitioned/2018/4/divvy_trips.csv')
s3.ObjectSummary(bucket_name='udacity-dend', key='data-pipelines/divvy/

#### Working with the Spark Code

In [4]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, unix_timestamp

In [5]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

#### Working with the song data

In [21]:
%%sh
unzip data/song-data.zip
mkdir data/song_data

Archive:  data/song-data.zip
   creating: song_data/
  inflating: song_data/.DS_Store     
   creating: song_data/A/
  inflating: song_data/A/.DS_Store   
   creating: song_data/A/A/
  inflating: song_data/A/A/.DS_Store  
   creating: song_data/A/A/A/
  inflating: song_data/A/A/A/TRAAAAW128F429D538.json  
  inflating: song_data/A/A/A/TRAAABD128F429CF47.json  
  inflating: song_data/A/A/A/TRAAADZ128F9348C2E.json  
  inflating: song_data/A/A/A/TRAAAEF128F4273421.json  
  inflating: song_data/A/A/A/TRAAAFD128F92F423A.json  
  inflating: song_data/A/A/A/TRAAAMO128F1481E7F.json  
  inflating: song_data/A/A/A/TRAAAMQ128F1460CD3.json  
  inflating: song_data/A/A/A/TRAAAPK128E0786D96.json  
  inflating: song_data/A/A/A/TRAAARJ128F9320760.json  
  inflating: song_data/A/A/A/TRAAAVG12903CFA543.json  
  inflating: song_data/A/A/A/TRAAAVO128F93133D4.json  
   creating: song_data/A/A/B/
  inflating: song_data/A/A/B/TRAABCL128F4286650.json  
  inflating: song_data/A/A/B/TRAABDL12903CAABBA.json  
  i

mv: cannot stat '*.json': No such file or directory


In [23]:
mv song_data data/

In [5]:
%%sh
find . -name '.DS_Store' -type f -delete
find data/song_data -name '.ipynb_checkpoints' -type f -delete

#### Loading Song Data

In [6]:
df_song_data = spark.read.json('data/song_data/*/*/*/')

In [7]:
df_song_data.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARDR4AC1187FB371A1,NaN,,NaN,Montserrat Caballé;Placido Domingo;Vicente Sar...,511.16363,1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,0
1,AREBBGV1187FB523D2,NaN,"Houston, TX",NaN,Mike Jones (Featuring CJ_ Mello & Lil' Bran),173.66159,1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),0
2,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
3,ARPBNLO1187FB3D52F,40.71455,"New York, NY",-74.00712,Tiny Tim,43.36281,1,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,2000
4,ARDNS031187B9924F0,32.67828,Georgia,-83.22295,Tim Wilson,186.48771,1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,2005


In [8]:
df_song_data.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



#### Preparing the songs table

songs - songs in music database
    
    song_id, title, artist_id, year, duration

In [9]:
df_songs = df_song_data.dropna(subset=['song_id', 'artist_id']) \
                    .select(['song_id', 'title', 'artist_id', 'year', 'duration'])

In [10]:
df_songs.limit(5).toPandas()

,song_id,title,artist_id,year,duration
0,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,ARDR4AC1187FB371A1,0,511.16363
1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),AREBBGV1187FB523D2,0,173.66159
2,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
3,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,ARPBNLO1187FB3D52F,2000,43.36281
4,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,ARDNS031187B9924F0,2005,186.48771


#### Preparing the artists table

artists - artists in music database
    
    artist_id, name, location, lattitude, longitude

In [11]:
df_artists = df_song_data.dropna(subset=['song_id', 'artist_id']) \
                    .select(['artist_id', 'artist_name', 'artist_location', \
                             'artist_latitude', 'artist_longitude'])

In [12]:
df_artists.limit(5).toPandas()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARDR4AC1187FB371A1,Montserrat Caballé;Placido Domingo;Vicente Sar...,,NaN,NaN
1,AREBBGV1187FB523D2,Mike Jones (Featuring CJ_ Mello & Lil' Bran),"Houston, TX",NaN,NaN
2,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
3,ARPBNLO1187FB3D52F,Tiny Tim,"New York, NY",40.71455,-74.00712
4,ARDNS031187B9924F0,Tim Wilson,Georgia,32.67828,-83.22295


**Writing to Parquet file**

In [14]:
# df_songs.write.parquet('songs', partitionBy=['year', 'artist_id'])

#### Unzipping and Loading the Log Data

In [10]:
%%sh
unzip data/log-data.zip
mkdir data/log_data
mv *.json data/log_data

Archive:  data/log-data.zip
  inflating: 2018-11-01-events.json  
  inflating: 2018-11-02-events.json  
  inflating: 2018-11-03-events.json  
  inflating: 2018-11-04-events.json  
  inflating: 2018-11-05-events.json  
  inflating: 2018-11-06-events.json  
  inflating: 2018-11-07-events.json  
  inflating: 2018-11-08-events.json  
  inflating: 2018-11-09-events.json  
  inflating: 2018-11-10-events.json  
  inflating: 2018-11-11-events.json  
  inflating: 2018-11-12-events.json  
  inflating: 2018-11-13-events.json  
  inflating: 2018-11-14-events.json  
  inflating: 2018-11-15-events.json  
  inflating: 2018-11-16-events.json  
  inflating: 2018-11-17-events.json  
  inflating: 2018-11-18-events.json  
  inflating: 2018-11-19-events.json  
  inflating: 2018-11-20-events.json  
  inflating: 2018-11-21-events.json  
  inflating: 2018-11-22-events.json  
  inflating: 2018-11-23-events.json  
  inflating: 2018-11-24-events.json  
  inflating: 2018-11-25-events.json  
  inflating: 2018-11-2

In [13]:
df_log_data = spark.read.json('data/log_data/')

In [14]:
df_log_data.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [15]:
df_log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
df_log_data = df_log_data.filter("page='NextSong'")

In [17]:
df_log_data.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


#### Preparing the Users table

users - users in the app

    user_id, first_name, last_name, gender, level

In [18]:
df_users = df_log_data.dropna(subset='userId') \
                .select([col('userId').alias('user_id'), col('firstName').alias('first_name'),\
                         col('lastName').alias('last_name'), 'gender', 'level'])

In [19]:
df_users.limit(5).toPandas()

,user_id,first_name,last_name,gender,level
0,26,Ryan,Smith,M,free
1,26,Ryan,Smith,M,free
2,26,Ryan,Smith,M,free
3,61,Samuel,Gonzalez,M,free
4,80,Tegan,Levine,F,paid


#### Preparing the time dimension table

time - timestamps of records in songplays broken down into specific units

    start_time, hour, day, week, month, year, weekday

In [20]:
df_log_data = df_log_data.drop("tsX")

In [82]:
df_log_data = df_log_data.withColumn('tsX', col('ts').cast('string'))

In [22]:
df_log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [84]:
df_log_data.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,tsX
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,1542241826796
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,1542242481796
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,1542242741796
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,1542253449796
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1542260935796


In [21]:
from pyspark.sql.functions import from_unixtime, to_timestamp, to_date

In [22]:
get_timestamp = udf(lambda ts: to_timestamp(from_unixtime(ts/1000, \
                                             format='yyyy-MM-dd HH:mm:ss')), TimestampType())

In [23]:
df_log_data = df_log_data.withColumn('tsX', to_timestamp(from_unixtime(df_log_data.ts/1000, \
                                             format='yyyy-MM-dd HH:mm:ss')))

# df_log_data = df_log_data.withColumn('tsX', get_timestamp('ts'))

In [24]:
df_log_data.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,tsX
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55


In [25]:
df_log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- tsX: timestamp (nullable = true)



In [26]:
from pyspark.sql.types import MapType, StringType, IntegerType
get_datetime = udf(lambda row: {'year': row.year, \
                                    'month': row.month,
                                    'week': row.isocalendar()[1],
                                    'day': row.day,
                                    'hour': row.hour,
                                    'weekday': row.weekday()
                                   },\
                   MapType(StringType(), IntegerType()))

In [27]:
df_log_data = df_log_data.withColumn('dt', get_datetime(df_log_data.tsX))

In [137]:
df_log_data.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,registration,sessionId,song,status,ts,userAgent,userId,tsX,year,dt
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26,2018,"{'weekday': 3, 'week': 46, 'hour': 0, 'month':..."
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21,2018,"{'weekday': 3, 'week': 46, 'hour': 0, 'month':..."
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41,2018,"{'weekday': 3, 'week': 46, 'hour': 0, 'month':..."
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,...,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61,2018-11-15 03:44:09,2018,"{'weekday': 3, 'week': 46, 'hour': 3, 'month':..."
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,...,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,2018-11-15 05:48:55,2018,"{'weekday': 3, 'week': 46, 'hour': 5, 'month':..."


In [28]:
df_log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- tsX: timestamp (nullable = true)
 |-- dt: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)



In [32]:
df_time = df_log_data.select([col('tsX').alias('start_time'), 'dt.hour', 'dt.day', 'dt.week', \
                              'dt.month', 'dt.year', 'dt.weekday'])

In [33]:
df_time.limit(5).toPandas()

,start_time,hour,day,week,month,year,weekday
0,2018-11-15 00:30:26,0,15,46,11,2018,3
1,2018-11-15 00:41:21,0,15,46,11,2018,3
2,2018-11-15 00:45:41,0,15,46,11,2018,3
3,2018-11-15 03:44:09,3,15,46,11,2018,3
4,2018-11-15 05:48:55,5,15,46,11,2018,3


In [74]:
df_song_data.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



#### Preparing the songplays table

songplays - records in log data associated with song plays i.e. records with page NextSong

    songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

In [34]:
cond = [(df_song_data.title==df_log_data.song), (df_song_data.artist_name==df_log_data.artist)]
df_join = df_log_data.join(df_song_data, cond, 'left')

In [35]:
df_join.count()

6820

In [36]:
df_log_data.count()

6820

In [37]:
df_join.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- tsX: timestamp (nullable = true)
 |-- dt: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: dou

**Note:** Using the monotonically_increasing_id to create songplays_id

In [38]:
from pyspark.sql.functions import monotonically_increasing_id
df_join = df_join.withColumn("songplay_id", monotonically_increasing_id() )

In [40]:
df_join.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year,songplay_id
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,None,None,None,None,None,None,None,None,None,0
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,None,None,None,None,None,None,None,None,None,1
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,None,None,None,None,None,None,None,None,None,2
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,...,None,None,None,None,None,None,None,None,None,3
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,...,None,None,None,None,None,None,None,None,None,4


In [41]:
df_join.head()

Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', tsX=datetime.datetime(2018, 11, 15, 0, 30, 26), dt={'weekday': 3, 'week': 46, 'hour': 0, 'month': 11, 'year': 2018, 'day': 15}, artist_id=None, artist_latitude=None, artist_location=None, artist_longitude=None, artist_name=None, duration=None, num_songs=None, song_id=None, title=None, year=None, songplay_id=0)

In [47]:
songplays = df_join.select('songplay_id', col('tsX').alias('start_time'), \
               col('userId').alias('user_id'), 'level', 'song_id', \
               'artist_id', col('sessionId').alias('session_id'), col('artist_location').alias('location'), \
               col('userAgent').alias('user_agent'))

In [48]:
songplays.limit(5).toPandas()

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,0,2018-11-15 00:30:26,26,free,None,None,583,None,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
1,1,2018-11-15 00:41:21,26,free,None,None,583,None,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
2,2,2018-11-15 00:45:41,26,free,None,None,583,None,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,3,2018-11-15 03:44:09,61,free,None,None,597,None,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,4,2018-11-15 05:48:55,80,paid,None,None,602,None,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
